## hand data synthesis
* 문제 정의
    * 웹캠에서 찍히는 구도에서 손을 segmentation 하는 문제로 좁힘
* 데이터 합성
    * 라벨있는 hand 데이터 (frei hand 등) + 다양한 백그라운드 및 얼굴 데이터 합성

In [1]:
from PIL import Image, ImageDraw
import numpy as np
from glob import glob
import os
import pandas as pd

In [2]:
def synthesis(bg_image, face_rgb_image, face_mask_image, hand_rgb_images, hand_mask_images):
    # load hand image
    rgb_images = list()
    mask_images = list()
    extracted_images = list()
    for hand_rgb_image, hand_mask_image in zip(hand_rgb_images, hand_mask_images):
        hand_rgb_array = np.array(hand_rgb_image)
        hand_mask_array = np.array(hand_mask_image)
        hand_extracted_array = hand_rgb_array * (hand_mask_array > 127)
        hand_extracted_image = Image.fromarray(hand_extracted_array)

        rgb_images.append(hand_rgb_image)
        mask_images.append(hand_mask_image)
        extracted_images.append(hand_extracted_image)

    face_rgb_array = np.array(face_rgb_image)
    face_mask_array = np.array(face_mask_image)
    face_extracted_array = face_rgb_array * (face_mask_array > 127)
    face_extracted_image = Image.fromarray(face_extracted_array)

    # hand transform
    for rgb, mask, ext in zip(rgb_images, mask_images, extracted_images):
        rotate = np.random.randint(-30, 30)
        resize = np.random.randint(50, 150)
        mask = mask.rotate(rotate).resize((resize, resize))
        ext = ext.rotate(rotate).resize((resize, resize))

    # face paste
    rotate = np.random.randint(-20, 20)
    resize = (np.random.randint(30, 50))
    face_rgb_image.rotate(rotate).resize((resize, resize))
    face_mask_image.rotate(rotate).resize((resize, resize))
    translate = (np.random.randint(0, 100), np.random.randint(-50, 50))
    bg_image.paste(face_extracted_image, translate, face_mask_image.convert('L'))
    
    # image, mask paste
    hand_adj_mask_image = Image.new('L', size=(bg_image.size))
    for ext, mask in zip(extracted_images, mask_images):
        translate = (np.random.randint(-100, 100), np.random.randint(-100, 100))
        bg_image.paste(ext, translate, mask.convert('L'))
        hand_adj_mask_image.paste(mask, translate, mask.convert('L'))

    # save img, mask
    return bg_image, hand_adj_mask_image

In [3]:
hand_base = r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\frei_hand\training\rgb'
hand_mask_base = r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\frei_hand\training\mask'
face_base = r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\CelebAMask-HQ\CelebA-HQ-img'
face_mask_base = r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\CelebAMask-HQ\CelebAMask-HQ-mask-anno'
bg_base = r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\House_Room_Dataset\Livingroom'
save_path = r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\synthesis_hand\train'

hand_list = glob(f'{hand_base}\*')
face_list = glob(f'{face_base}\*')
bg_list = glob(f'{bg_base}\*')

len_data = len(glob(r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\synthesis_hand\train\*')) * 2

for i in range(10000):
    try:
        # background
        bg_rgb = bg_list[np.random.randint(len(bg_list))]
        bg_image = Image.open(bg_rgb)

        # face
        face_rgb = face_list[np.random.randint(len(face_list))]
        picked = ('0'*5 + (face_rgb.split('\\')[-1].split('.')[0]))[-5:]
        direc = int((int(picked)+1) // 2000)
        face_rgb_image = Image.open(face_rgb)
        face_mask = face_rgb.replace('CelebA-HQ-img', f'CelebAMask-HQ-mask-anno\\{direc}')
        face_id = face_mask.split('\\')[-1].split('.')[0]
        face_id_fill = ('0'*5 + face_id)[-5:]
        face_mask = face_mask.replace('.jpg', '').replace(f'{face_id}', f'{face_id_fill}')
        face_mask_pathes = glob(f'{face_mask}*')
        face_mask_image = Image.open(face_mask_pathes[0])
        for m in face_mask_pathes[1:]:
            face_mask_image.paste(Image.open(m), (0, 0), Image.open(m).convert('L'))
        face_rgb_image = face_rgb_image.resize((128, 128))
        face_mask_image = face_mask_image.resize((128, 128))

        # hands
        hand_rgb_images = list()
        hand_mask_images = list()
        n_hands = np.random.randint(3)
        for h in range(n_hands):
            hand_rgb = hand_list[np.random.randint(len(hand_list))]
            hand_id = int(hand_rgb.split('\\')[-1].split('.')[0]) % 32560
            hand_id = ('0'*8 + str(hand_id))[-8:]
            hand_mask = os.path.join(hand_rgb.split('\\rgb')[0], 'mask', f'{hand_id}.jpg')
            hand_rgb_images.append(Image.open(hand_rgb))
            hand_mask_images.append(Image.open(hand_mask))

        synth, mask = synthesis(bg_image, face_rgb_image, face_mask_image, hand_rgb_images, hand_mask_images)
        synth.save(os.path.join(save_path, f'image_{i+len_data}.jpg'))
        mask.save(os.path.join(save_path, f'mask_{i+len_data}.jpg'))
    except:
        pass

In [ ]:
synth

In [ ]:
mask

In [3]:
# make lut
save_path = r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\synthesis_hand\train'

df = pd.DataFrame()
total_datas = glob(f'{save_path}\*')

images = [d for d in total_datas if 'image' in d]
masks = list() #[d for d in total_datas if 'mask' in d]

for img in images:
    mask = img.replace('image', 'mask')
    masks.append(mask)

df['image'] = images
df['mask'] = masks

df.to_csv(r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\synthesis_hand\lut.csv')

In [11]:
import shutil
shutil.rmtree(r'\\wsl.localhost\Ubuntu-20.04\home\hebb\ml\datasets\synthesis_hand\train')